El siguiente código ha sido desarrollado por IAUAI y se encuentra con licencia de código abierto, así que siéntete libre de usarlo y modificarlo a tu gusto ✌

TorToiSe es un modelo para generar audio en base a texto (*TTS*). El particular atractivo que tiene es la capacidad de replicar de forma sencilla una voz humana. No obstante, sólo trabaja con voces y texto en inglés. También es importante notar que tiene un límite en la extensión del texto que puede generar por audio, por lo que, para parchar eso, este código segmenta tu texto en oraciones (lo cual implica que es probable que veas que se genere más de un audio).

Necesitarás 2 cosas para usar este código:
1.   Al menos 5 muestras de la voz que quieres replicar en formato .wav, cuya duración no supere los 10 segundos. Entre más muestras, mejor, pero asegúrate de seleccionar audios libres de ruidos indeseados o de "anomalías" (como muletillas o tartamudeos) que no quieres que sean aprendidas por el modelo.
2.   El texto que quieres que sea leido. Puedes ingresarlo a mano, mas, si es extenso, puede ser más aconsejable el aprovechar la opción de subir un archivo .txt.

Este código está pensado para ser usado en Google Colab (podría emplearse localmente, aunque se deben hacer ajustes al mismo para que funcione). También es áltamente recomendado que sea corrido en GPU para que no tarde horas en ejecutarse e, incluso con la GPU habilitada, no te sorprendas si le toma varios minutos en generar el audio (he ahí la razón del nombre del modelo 🐢).

Repositorio de este código, donde puedes hallar un ejemplo y darnos tu feedback: https://github.com/IA-UAI/TorToiSe

Repositorio del modelo: https://github.com/neonbjb/tortoise-tts

Código usado como referencia: https://colab.research.google.com/drive/1wVVqUPqwiDBUVeWWOUNglpGhU3hg_cbR?usp=sharing#scrollTo=fYTk8KUezUr5

In [ ]:
#@title Instalaciones, importaciones y preparación general

#@markdown Primero hay que hacer las preparaciones para usar el modelo. Tras la primera ejecución, puedes volver a correr esta celda para añadir nuevas voces (nada más asegúrate de darle un nombre distinto).
#@markdown Por otro lado, si bien puede ser un poco más tedioso el subir las muestras a mano, es recomendable que lo hagas así cuando tengas más de 10, ya que de esa forma se suben mucho más rápido.

import os
if (os.getcwd() == "/content"):
  !pip3 install -U scipy
  !git clone https://github.com/jnordberg/tortoise-tts.git
  %cd tortoise-tts
  !pip3 install transformers==4.19.0  !pip3 install -r requirements.txt
  !python3 setup.py install

  from IPython.display import clear_output
  import torch
  import torchaudio
  import torch.nn as nn
  import torch.nn.functional as F
  from IPython.display import Audio, display
  from tortoise.api import TextToSpeech
  from tortoise.utils.audio import load_audio, load_voice, load_voices
  from google.colab import files
  import shutil
  import time
  from datetime import timedelta
  tts = TextToSpeech()
clear_output()
nombre_de_la_nueva_voz = "nueva_voz" #@param {type:"string"}
subir_a_mano = False #@param {type:"boolean"}
custom_voice_folder = f"tortoise/voices/{nombre_de_la_nueva_voz}"
if (os.path.isdir(custom_voice_folder)):
  print("Esta voz ya existe. Prueba con otro nombre.")
else:
  os.makedirs(custom_voice_folder)
  if (subir_a_mano): print("Sube las muestras a","tortoise-tts/"+custom_voice_folder)
  else:
    uploaded = files.upload()
    for muestra in uploaded.keys():
      shutil.move("./"+muestra, custom_voice_folder)

In [ ]:
#@title Selección de texto

#@markdown Ahora toca el seleccionar el texto. Puedes subir uno o varios archivos .txt. No obstante, si subes varios, luego debes asegurarte de escoger cuál usar. Y, si así lo prefieres, también puedes seleccionar la opción de escribir texto para introducir tu texto a mano en la consola.

metodo_de_seleccion_de_texto = "Subir y seleccionar nuevo texto" #@param ["Subir y seleccionar nuevo texto", "Seleccionar texto subido", "Escribir texto"]
maximo_de_caracteres_a_mostrar = 50 #@param {type:"integer"}
maximo_de_fragmentos_a_mostrar = 3 #@param {type:"integer"}
texto = ""
if (metodo_de_seleccion_de_texto == "Subir y seleccionar nuevo texto"):
  print("Suba el archivo .txt")
  uploaded = files.upload()
  dir = list(uploaded.keys())[0]
  f = open(dir, "r")
  texto = f.read()
  f.close()
  print("\n#################### Este es el inicio de tu texto a continuación ####################\n")
  print(texto[:maximo_de_caracteres_a_mostrar])
elif (metodo_de_seleccion_de_texto == "Seleccionar texto subido"):
  print("Ingrese el índice del archivo que seleccionar")
  opciones = [i for i in os.listdir("./") if (i[-4:] == ".txt") and (i != "requirements.txt")]
  if (len(opciones) > 0):
    indices = [i+1 for i in range(len(opciones))]
    res = dict(zip(indices, opciones))
    for i in res: print(str(i)+":", res[i])
    texto = input()
    try:
      texto = res[int(texto)]
      f = open(texto, "r")
      texto = f.read()
      f.close()
      print("\n#################### Este es el inicio de tu texto a continuación ####################\n")
      print(texto[:maximo_de_caracteres_a_mostrar])
    except:
      print("Ocurrió un error. Escriba el número sin caracteres adicionales.")
  else: print("No hay archivos disponibles. Debes subir uno.")
else:
  print("Ingrese su texto")
  texto = input()
  print("\n#################### Este es el inicio de tu texto a continuación ####################\n")
  print(texto[:maximo_de_caracteres_a_mostrar])

while ("\n " in texto): texto = texto.replace("\n ", "\n")
while (" \n" in texto): texto = texto.replace(" \n", "\n")
while ("\n\n" in texto): texto = texto.replace("\n\n", "\n")
while (".\n" in texto): texto = texto.replace(".\n", ".")
while ("  " in texto): texto = texto.replace("  ", " ")
while (". " in texto): texto = texto.replace(". ", ".")
while (",\n" in texto): texto = texto.replace(",\n", "\n")
while (":\n" in texto): texto = texto.replace(":\n", "\n")
while (";\n" in texto): texto = texto.replace(";\n", "\n")
while ("\n" in texto): texto = texto.replace("\n", ".")
fragmentos = [i for i in texto.split(".") if (i != "")]
print("\n#################### Ejemplos de fragmentos a continuación ####################\n")
for i in fragmentos[:maximo_de_fragmentos_a_mostrar]: print(i)

In [ ]:
#@title Generación de audios

#@markdown Momento de la generación. Aquí queda a tu criterio qué tanta calidad quieres en tu generación (más calidad significa más realismo y semejanza a la voz de referencia, pero también más tiempo para generar los audios).

voice_samples, conditioning_latents = load_voice(nombre_de_la_nueva_voz)
tipo_de_calidad = "Alta_calidad" #@param ["Generacion_ultra_rapida", "Generacion_rapida", "Calidad_estandar", "Alta_calidad"]
if (tipo_de_calidad == "Alta_calidad"): tipo_de_calidad = "high_quality"
elif (tipo_de_calidad == "Calidad_estandar"): tipo_de_calidad = "standard"
elif (tipo_de_calidad == "Generacion_rapida"): tipo_de_calidad = "fast"
elif (tipo_de_calidad == "Generacion_ultra_rapida"): tipo_de_calidad = "ultra_fast"

contador = 1
ver_el_avance = True #@param {type:"boolean"}
descargar_audios = False #@param {type:"boolean"}
start_time = time.monotonic()
for fragmento in fragmentos:
  gen = tts.tts_with_preset(fragmento, voice_samples=voice_samples, conditioning_latents=conditioning_latents, 
                            preset=tipo_de_calidad, verbose=ver_el_avance)
  torchaudio.save(f'{nombre_de_la_nueva_voz}_{contador}.wav', gen.squeeze(0).cpu(), 24000)#, encoding="PCM_F")
  print(contador,"de",len(fragmentos),"|",f'{nombre_de_la_nueva_voz}_{contador}.wav')
  display(Audio(f'{nombre_de_la_nueva_voz}_{contador}.wav', autoplay=False))
  contador += 1
end_time = time.monotonic()
print("Tiempo tomado en generar audio(s):", timedelta(seconds=end_time - start_time))

if (descargar_audios):
  for i in range(1,contador):
    files.download(f'{nombre_de_la_nueva_voz}_{i}.wav')

In [ ]:
#@title Unir audios

#@markdown Finalmente, aquí tienes la opción de unir los audios que has generado.

infiles = [f'{nombre_de_la_nueva_voz}_{i}.wav' for i in range(1,contador)]
outfile = f'{nombre_de_la_nueva_voz}_final.wav'

data = torch.tensor([])
for infile in infiles:
  data = torch.concat([data, torchaudio.load(infile)[0].reshape(-1)])

torchaudio.save(f'{nombre_de_la_nueva_voz}_final.wav', data.reshape(1,-1), 24000)

display(Audio(f'{nombre_de_la_nueva_voz}_final.wav', autoplay=False))

descargar_audio_final = True #@param {type:"boolean"}
if (descargar_audio_final): files.download(f'{nombre_de_la_nueva_voz}_final.wav')